In [1]:
from hmmlearn import hmm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

In [47]:
X = pd.DataFrame(pd.pandas.read_csv("../data/3Boxes_coords.csv", names=['Gauche_head', 'Haut_head', 'Largeur_head', 'Longeur_head', 'Gauche_snout', 'Haut_snout', 'Largeur_snout', 'Longeur_snout', 'Gauche_cow', 'Haut_cow', 'Largeur_cow', 'Longeur_cow']))
X = X.drop(index=X.index[0])
model_3boxes = joblib.load('models/EM/hmm_model_head_snout_cow.pkl')
hidden_states = model_3boxes.predict(X)
hidden_states_count = pd.DataFrame(hidden_states, columns=['values'])
# print(hidden_states_count)
hidden_states_counted = hidden_states_count.groupby([hidden_states_count['values'].ne(hidden_states_count['values'].shift()).cumsum(), 'values']).size()

data = {
    'state': [0, 3, 2, 1, 2, 1, 2],
    'count': [484, 61, 111, 23, 187, 464, 45]
}
df = pd.DataFrame(data)

df['frame'] = df['count'].cumsum()
df['seconds'] = df['frame']/30

print(df)

   state  count  frame    seconds
0      0    484    484  16.133333
1      3     61    545  18.166667
2      2    111    656  21.866667
3      1     23    679  22.633333
4      2    187    866  28.866667
5      1    464   1330  44.333333
6      2     45   1375  45.833333


In [66]:
behavior = pd.pandas.read_csv("E:/UQAM/INF889E/projet/Recherche/Data/emotion_analyses/emotion_analyses/NOT_Lou_annotations.csv", header=None)

new_columns = behavior[0].str.split(',', expand=True)

behavior[new_columns.columns] = new_columns

behavior = behavior.drop(columns=[0])
temps_annotation = behavior[[5, 11]].drop(behavior.index[0])
annotation = pd.DataFrame(temps_annotation)
annotation = annotation.replace("00:00:", "", regex=True)

print(type(annotation[5]))

<class 'pandas.core.series.Series'>


In [ ]:
for i in range(0, len(df)):
    for j in range(0, len(annotation)):
        if j <= df['frame']:
            df['behavior'] = df['behavior'].append(annotation[5])
# LINAAAAAA u finish it today